# **Simply supported beam as a plate**

In this example we model a simply supported beam as a plate, under the effect of a constant distributed load. We run run a linear analysis and do some plotting.

***

### **Create a COM interface to a new instance of AxisVM**

We create a new instance of AxisVM, and make it visible. For the meaning of the argument `daemon=True`, take a look on the docstring of the function.

In [69]:
from axisvm.com.client import start_AxisVM
axvm = start_AxisVM(visible=True, daemon=True)


The type library itself can be imported as

In [70]:
import axisvm.com.tlb as axtlb

If this is not the first time, this import statement can be at the top of the unit.

Create a new model and set the working directory. An empty string means the directory where AxisVM.exe is located:

In [71]:
modelId = axvm.Models.New()
axm = axvm.Models.Item[modelId]
wdir = ""


### **Input data**

When talking about input data, don't forget, that AxisVM internally stores values in $kN$ and $m$.

In [72]:
L = 10.0  # length of the beam
h = 2.0  # height of the cross-section
b = 0.2  # width of the cross section
pz = -10.0/b  # intensity of vertical distributed load
px = 0.0  # intensity of horizontal distributed load


### **Material**

Set Eurocode as the standard and "C16/20" concrete form the material library.

In [73]:
axm.Settings.NationalDesignCode = axtlb.ndcEuroCode
matId = axm.Materials.AddFromCatalog(axtlb.ndcEuroCode, "C16/20")


### **Geometry**

The definition of a geometry of a model follows a hierarchical workflow. This means, that we cannot directly define the domains. Instead, we first define the nodes that make up the lines, with which finally we define the domains.

We create the beam as two touching domains, which makes up for a total of 6 nodes. We also store the indices of the defined nodes as a reference for creating the lines.

In [74]:
import numpy as np
from axisvm.com.tlb import dofPlateXY

# the plate is in the x-y plane
coords = np.zeros((4, 3))  # we have four points in 3d space
coords[0, :] = 0., -b/2, 0.
coords[1, :] = L, -b/2, 0.
coords[2, :] = L, b/2, 0.
coords[3, :] = 0., b/2, 0.

fnc = axm.Nodes.AddWithDOF
nodeIDs = list(map(lambda c: fnc(*c, dofPlateXY), coords))


We define the lines that make up the domains and store their indices in a `list`. 

In [75]:
nodes_of_lines = [[0, 1], [1, 2], [2, 3], [3, 0]]
LineGeomType = axtlb.lgtStraightLine
lineIDs = []
for line in nodes_of_lines:
    lineIDs.append(axm.Lines.Add(nodeIDs[line[0]], nodeIDs[line[1]],
                   LineGeomType, axtlb.RLineGeomData())[1])


Now we are in a position to create the domains by providing lineIDs.

In [76]:
from axisvm.com.tlb import RSurfaceAttr, lnlTensionAndCompression, \
    RResistancesXYZ, schLinear, stPlate, RElasticFoundationXYZ, \
    RNonLinearityXYZ

sattr = RSurfaceAttr(
    Thickness=h,
    SurfaceType=stPlate,
    RefZId=0,
    RefXId=0,
    MaterialId=matId,
    ElasticFoundation=RElasticFoundationXYZ(0, 0, 0),
    NonLinearity=RNonLinearityXYZ(lnlTensionAndCompression,
                                  lnlTensionAndCompression,
                                  lnlTensionAndCompression),
    Resistance=RResistancesXYZ(0, 0, 0),
    Charactersitics=schLinear)

_ = axm.Domains.Add(LineIds=lineIDs, SurfaceAttr=sattr)


### **Loads**

We add a line load on the whole span of the beam with the specified load intensities.

In [77]:
LoadDomainConstant = axtlb.RLoadDomainConstant(
    LoadCaseId=1,
    DomainId=1,
    qx=0,
    qy=0,
    qz=pz,
    DistributionType=axtlb.sddtSurface,
    SystemGLR=axtlb.sysGlobal
)
_ = axm.Loads.AddDomainConstant(LoadDomainConstant)


### **Supports**

We add two hinged supports at the ends.

In [78]:
springleft = axtlb.RStiffnesses(
    x=1e12,
    y=1e12,
    z=1e12,
    xx=0,
    yy=0,
    zz=0
)
springright = axtlb.RStiffnesses(
    x=1e12,
    y=1e12,
    z=1e12,
    xx=0,
    yy=0,
    zz=0
)
RNonLinearity = axtlb.RNonLinearity(
    x=axtlb.lnlTensionAndCompression,
    y=axtlb.lnlTensionAndCompression,
    z=axtlb.lnlTensionAndCompression,
    xx=axtlb.lnlTensionAndCompression,
    yy=axtlb.lnlTensionAndCompression,
    zz=axtlb.lnlTensionAndCompression
)
RResistances = axtlb.RResistances(
    x=0,
    y=0,
    z=0,
    xx=0,
    yy=0,
    zz=0
)
_ = axm.LineSupports.AddEdgeGlobal(springleft, RNonLinearity,
                                   RResistances, 2, 0, 0, 1, 0)
_ = axm.LineSupports.AddEdgeGlobal(springright, RNonLinearity,
                                   RResistances, 4, 0, 0, 1, 0)


### **Meshing**

We define a triangle mesh, with a mesh size of $h/2$.

In [79]:
MeshParams = axtlb.RDomainMeshParameters(
    MeshSize=b/2,
    MeshType=axtlb.mtUniform,
    MeshGeometryType=axtlb.mgtTriangle
)
axm.Domains[:].GenerateMesh(MeshParams);

Notice the use of the semicolon here. This simplifies carrying out the same operation over a range of domains. (the colon at the end simply suppresses the output).

### **Processing**

We save the file and run a linear analysis, with all warnings suppressed.

In [80]:
fpath = wdir + 'ss_beam_P.axs'
axm.SaveToFile(fpath, False)
axm.Calculation.LinearAnalysis(axtlb.cuiNoUserInteractionWithAutoCorrectNoShow)


1

### **Plotting with AxisVM**

We want to plot vertical displacements. At the end we save the plot to a file in the working directory.

In [81]:

# we want the domain to fill up the screen entirely
axm.View = axtlb.vTop
axm.FitInView()

# turn off the grid
GridOptions = axtlb.RGridOptions(DisplayGrid=False)
axm.Settings.SetGridOptions(GridOptions)

WriteValuesTo = axtlb.RWriteValuesTo(
    Nodes=True,
    Lines=True,
    Surfaces=False,
    MinMaxOnly=True
)

BasicDispParams = axtlb.RBasicDisplayParameters_V153(
    ResultComponent=axtlb.rc_d_eZ,
    Scale=1.0,
    DisplayMode=axtlb.dmIsosurfaces2D,
    DisplayShape=axtlb.dsUndeformed,
    WriteValuesTo=WriteValuesTo
)

ExtDispParams = axtlb.RExtendedDisplayParameters_V153(
    BasicDispParams=BasicDispParams,
    DisplayAnalysisType=axtlb.datLinear,
    ResultType=axtlb.rtLoadCase
)
axm.Windows.SetStaticDisplayParameters_V153(1, ExtDispParams, 1, [])

axm.Windows.ReDraw()
imgpath = wdir + 'ss_beam_P_ez.bmp'
axm.Windows[1].SaveWindowToBitmap(axtlb.wcmColour, imgpath)
axvm.BringToFront()


1

### **Close AxisVM**

Because we created the interface with `daemon=True`, the application closes without any warning.

In [68]:
del axvm